## Deliverable 1.  Retrieve Weather Data

In [1]:
# Import all necessary libraries and the openweathermap API key
import numpy as np
import pandas as pd
import requests
from citipy import citipy
from datetime import datetime
import time

from config import openweather_api_key

In [2]:
# Create the base url, including the API key and request for imperial (US) units
url = "http://api.openweathermap.org/data/2.5/weather?" + "appid=" + openweather_api_key + "&units=imperial"

In [3]:
# Create a set of random latitude and longitude combinations.
num_lat_longs = 2000
lats = np.random.uniform(low=-90.000, high=90.000, size=num_lat_longs)
lngs = np.random.uniform(low=-180.000, high=180.000, size=num_lat_longs)
coordinates = zip(lats, lngs)


In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
print(f'Number of cities:  {len(cities)}')


Number of cities:  733


## Download the weather data from openweatherapi.org; this will take several minutes due to limitations with the free subscription

In [5]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
# Capture the start time, this is used later to make sure we do not
# interrogate the API more than 60 times per second
start_time = time.time()
for i, city in enumerate(cities):
    
    # Group cities in sets of 60 for logging and API call management.
    if (i % 60 == 0 and i >= 60):
        set_count += 1
        record_count = 1
        
        # If this series of 60 calls took less than 60 seconds, then wait until
        # 61 seconds has elapsed (add 1 second margin)
        curr_time = time.time()
        elapsed_time = curr_time - start_time
        if elapsed_time < 61:
            wait_time = 61 - elapsed_time
            print (f"Waiting {wait_time:.0f} seconds for next API time slot...")
            time.sleep(wait_time)
            # Capture a new start time for the next block
            start_time = time.time()
            
    # Create endpoint URL for the current city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for the current city
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_desc = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description" : city_weather_desc})

    except KeyError:
        # A KeyError exception is thrown at the line "city_lat = ..."  if the
        #  API does not return any city data.
        print(city + " not found. Skipping...")
        pass
    
    except BaseException as ex:
        # Some other exception occurred, print it to the log
        print(f'Unknown exception:  {ex}')    

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | upernavik
Processing Record 2 of Set 1 | norman wells
Processing Record 3 of Set 1 | sosnogorsk
Processing Record 4 of Set 1 | cape town
Processing Record 5 of Set 1 | busselton
Processing Record 6 of Set 1 | barrow
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | saint-philippe
Processing Record 10 of Set 1 | acapulco
Processing Record 11 of Set 1 | avarua
Processing Record 12 of Set 1 | ormara
Processing Record 13 of Set 1 | kaeo
Processing Record 14 of Set 1 | meadow lake
Processing Record 15 of Set 1 | hasaki
Processing Record 16 of Set 1 | qaqortoq
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | illoqqortoormiut
illoqqortoormiut not found. Skipping...
Processing Record 19 of Set 1 | kailua
Processing Record 20 of Set 1 | marsa matruh
Processing Record 21 of Set 1 | porto murtinho
Processing Record 22 of Set 1

Processing Record 2 of Set 4 | saint pete beach
Processing Record 3 of Set 4 | angoche
Processing Record 4 of Set 4 | lebu
Processing Record 5 of Set 4 | nizhnyaya pavlovka
nizhnyaya pavlovka not found. Skipping...
Processing Record 6 of Set 4 | bluff
Processing Record 7 of Set 4 | temaraia
temaraia not found. Skipping...
Processing Record 8 of Set 4 | yulara
Processing Record 9 of Set 4 | yar-sale
Processing Record 10 of Set 4 | nome
Processing Record 11 of Set 4 | leningradskiy
Processing Record 12 of Set 4 | saint-georges
Processing Record 13 of Set 4 | shawville
Processing Record 14 of Set 4 | xining
Processing Record 15 of Set 4 | bandarbeyla
Processing Record 16 of Set 4 | saldanha
Processing Record 17 of Set 4 | aksarka
Processing Record 18 of Set 4 | castro
Processing Record 19 of Set 4 | batemans bay
Processing Record 20 of Set 4 | clinton
Processing Record 21 of Set 4 | provost
Processing Record 22 of Set 4 | rodeo
Processing Record 23 of Set 4 | viligili
viligili not found. 

Processing Record 5 of Set 7 | ahuimanu
Processing Record 6 of Set 7 | cockburn town
Processing Record 7 of Set 7 | gilgit
Processing Record 8 of Set 7 | sorland
Processing Record 9 of Set 7 | sitka
Processing Record 10 of Set 7 | satitoa
satitoa not found. Skipping...
Processing Record 11 of Set 7 | khatanga
Processing Record 12 of Set 7 | santa cruz
Processing Record 13 of Set 7 | cayenne
Processing Record 14 of Set 7 | flinders
Processing Record 15 of Set 7 | neosho
Processing Record 16 of Set 7 | haren
Processing Record 17 of Set 7 | inhambane
Processing Record 18 of Set 7 | karratha
Processing Record 19 of Set 7 | pangnirtung
Processing Record 20 of Set 7 | sapanta
Processing Record 21 of Set 7 | celestun
Processing Record 22 of Set 7 | ternate
Processing Record 23 of Set 7 | fort nelson
Processing Record 24 of Set 7 | roald
Processing Record 25 of Set 7 | parthenay
Processing Record 26 of Set 7 | derventa
Processing Record 27 of Set 7 | reconquista
Processing Record 28 of Set 7 |

Processing Record 17 of Set 10 | storforshei
Processing Record 18 of Set 10 | kastamonu
Processing Record 19 of Set 10 | fussa
Processing Record 20 of Set 10 | kiama
Processing Record 21 of Set 10 | calbuco
Processing Record 22 of Set 10 | catamarca
Processing Record 23 of Set 10 | dawson creek
Processing Record 24 of Set 10 | biltine
Processing Record 25 of Set 10 | fairbanks
Processing Record 26 of Set 10 | yambio
Processing Record 27 of Set 10 | dawlatabad
Processing Record 28 of Set 10 | aljezur
Processing Record 29 of Set 10 | iquitos
Processing Record 30 of Set 10 | westport
Processing Record 31 of Set 10 | tabiauea
tabiauea not found. Skipping...
Processing Record 32 of Set 10 | tigil
Processing Record 33 of Set 10 | umzimvubu
umzimvubu not found. Skipping...
Processing Record 34 of Set 10 | urumqi
Processing Record 35 of Set 10 | hede
Processing Record 36 of Set 10 | beisfjord
Processing Record 37 of Set 10 | saint-joseph
Processing Record 38 of Set 10 | hamilton
Processing Rec

-----------------------------
Data Retrieval Complete      
-----------------------------


In [6]:
# Save the city data to a CSV
city_data_df = pd.DataFrame(city_data)
city_data_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 
                          'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[city_data_column_order]
output_data_file = "WeatherPy_Database.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [7]:
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Upernavik,GL,72.7868,-56.1549,25.41,64,0,2.55,clear sky
1,Norman Wells,CA,65.2820,-126.8329,33.84,80,75,12.66,broken clouds
2,Sosnogorsk,RU,63.6023,53.8817,39.81,77,100,9.46,overcast clouds
3,Cape Town,ZA,-33.9258,18.4232,62.28,74,0,3.00,clear sky
4,Busselton,AU,-33.6500,115.3333,57.04,90,76,14.36,broken clouds
...,...,...,...,...,...,...,...,...,...
670,Najran,SA,17.4924,44.1277,83.05,19,10,5.50,clear sky
671,Zhigansk,RU,66.7697,123.3711,30.79,93,15,16.22,few clouds
672,Lianzhou,CN,24.7811,112.3825,77.61,80,45,0.96,scattered clouds
673,Mangla,PK,31.8931,72.3817,88.83,39,0,6.82,clear sky
